In [23]:
from termcolor import cprint
import sys

In [24]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

In [25]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            return self.frontier.pop(-1)

class QueueFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    
    def empty(self):
        return len(self.frontier) == 0

    def size(self):
        return len(self.frontier)
        
    def pop(self):
        if self.empty():
            raise Exception("fontier is empty")
        else:
            return self.frontier.pop(0)

In [26]:
# Blind mazes (No knowledge)
# Switch between Stack and Queue Frontiers for Depth First and Breadth First algos respectively

In [27]:
class Maze():
    def __init__(self, filename:str):
        # Read file
        with open(filename, 'r') as f:
            contents = f.read()
        
        if contents.count('A') != 1:
            raise Exception("Maze needs a single startpoint")
        if contents.count('B') != 1:
            raise Exception("Maze needs a single endpoint")

        contents = contents.splitlines()
        # Record height and width of maze
        self.height = len(contents)
        self.width = max(len(line) for line in contents)

        # Add context
        self.walls = [] # 2D bool array
        for r in range(self.height):
            row = []
            for c in range(self.width):
                # Try block as not all mazes will be rectangular
                try:
                    if contents[r][c] == '#':
                        row.append(True)
                    elif contents[r][c] == ' ':
                        row.append(False)
                    elif contents[r][c] == 'A':
                        self.start = (r,c)
                        row.append(False)
                    elif contents[r][c] == 'B':
                        self.goal = (r,c)
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(True)
            self.walls.append(row)
        self.solution = None

In [28]:
class Agent():
    def __init__(self, frontier):
        self.current_state = None
        self.explored = set()
        self.frontier = frontier
    
    def neighbors(self, maze:Maze):
        row, col = self.current_state
        # All possible actions
        candidates = [
            ('up', (row-1, col)),
            ('down', (row+1, col)),
            ('left', (row, col-1)),
            ('right', (row, col+1))
        ]
        # Validate actions
        possible = []
        for action, (row, col) in candidates:
            try:
                if not maze.walls[row][col]:
                    possible.append((action, (row,col)))
            except IndexError:
                continue
        return possible

    def solve(self, maze:Maze):
        # Keep track of how many states have been explored (performance measure)
        # self.num_explored = 0

        # Initialize frontier to the starting position
        start = Node(state=maze.start, parent=None, action=None)
        self.frontier.add(start)


        # Search for solution
        while True:
            if self.frontier.empty():
                cprint('No solution', 'red')
                break
            node = self.frontier.pop()
            self.current_state = node.state
            if self.current_state == maze.goal:
                # Add to explored
                self.explored.add(self.current_state)
                cprint('Solved!', 'green')
                actions = []
                cells = []

                # Follow breadcrumbs to remake solution path
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                # B -> A --> A -> B
                actions.reverse()
                cells.reverse()
                maze.solution = (actions, cells)
                return
            # Add to explored
            self.explored.add(self.current_state)

            # Add neighbors to frontier
            for action, state in self.neighbors(maze):
                if not self.frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    self.frontier.add(child)
            
    def print_solution(self, maze:Maze):
        for row in range(maze.height):
            for col in range(maze.width):
                if (row,col) == maze.start:
                    cprint('A', 'cyan', end='')
                elif (row,col) == maze.goal:
                    cprint('B', 'cyan', end='')
                elif maze.walls[row][col] == True:
                    print('#', end='')
                elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                    cprint('*', 'green', end='')
                elif (row,col) in self.explored:
                    cprint('e', 'red', end='')
                else:
                    print(' ', end='')
            print()
        print()

        actions, cells = maze.solution
        print('States explored: ' + str(len(self.explored)))
        print('Length of solution: ' + str(len(actions)))

    def print_solution(self, maze:Maze, fog_of_war:bool = False):
        for row in range(maze.height):
            for col in range(maze.width):
                if fog_of_war:
                    if (row,col) in self.explored:
                        if (row,col) == maze.start:
                            cprint('A', 'cyan', end='')
                        elif (row,col) == maze.goal:
                            cprint('B', 'cyan', end='')
                        elif maze.walls[row][col] == True:
                            print('#', end='')
                        elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                            cprint('*', 'green', end='')
                        elif (row,col) in self.explored:
                            cprint('e', 'red', end='')
                        else:
                            print(' ', end='')
                    else:
                        print('?', end='')
                else:
                    if (row,col) == maze.start:
                        cprint('A', 'cyan', end='')
                    elif (row,col) == maze.goal:
                        cprint('B', 'cyan', end='')
                    elif maze.walls[row][col] == True:
                        print('#', end='')
                    elif maze.solution is not None and (row,col) in maze.solution[1]: ##########
                        cprint('*', 'green', end='')
                    elif (row,col) in self.explored:
                        cprint('e', 'red', end='')
                    else:
                        print(' ', end='')
            print()
        print()

        actions, cells = maze.solution
        print('States explored: ' + str(len(self.explored)))
        print('Length of solution: ' + str(len(actions)))



In [34]:
mazer = Maze('maze3.txt')
ration = Agent(QueueFrontier())
ration.solve(mazer)
ration.print_solution(mazer, fog_of_war=False)

Solved!
#########################################################################
#eeeeeeeeeeeeeeeeeeeeeee#eeeeeeeeeeeeeeeeeeee#eeeeee#eeeeeeeeeee#########
#ee#####e##eeeeee##eeeeeeeeee#########eeeeeee#eeeeee#eeeeeeeeeee#########
##e#e###ee#ee#e#eeeeee####e###############eee#eeeeee#eeeeeeee#ee#########
#ee#e####eeee#eeee#eeeeeeeeeeee######eeeeee###eeeeeeeeeeeeeeee#ee########
##eeee##ee#eeee#####eeee################ee####eeeeeeeeeeeeeeeee#ee#######
####eeeeee#e#eeeee#ee#eeeee###eeeeeeeeeeeeeee###################ee#######
#eeeee#####e#eeeeee#eeee#eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee#
#eeeeeeeeeeee#eeeeeeeeeeeeee#eee##########eee###########eee##############
#eeeeeeeeeeee##########eeeeeeeeeee######eeeee#        #eee###############
#ee###e#e##eeeeeee#******ee##eeeeeee#eeeeeeee#       #eee################
##e#eee#ee#ee#eeee#*eee#***ee#eeeeeeeeeeeeeee#      #eee#################
#ee#e####eeee#******ee####*e##########eeeeeee#     #eee##################
##eeee##ee#****#####ee#eee*eee

In [30]:
def smaller(a:int, b:int):
    return a*(a<b) + b*(b<=a)

In [31]:
smaller(5,3)

3